**Natural Language Processing of User Queries, Alerts, and Reminders**

In [ ]:
!pip install spacy
import spacy
import re
from datetime import datetime

In [ ]:
'''
Natural Language Processing of User Queries
'''

# Word to Number Processing
def word_to_number(word):
    word_dict = {
        'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5,
        'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10
    }
    return word_dict.get(word.lower(), word)

# Load the English language model
nlp = spacy.load("en_core_web_sm")

def extract_relevant_info(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Current time in GMT
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Define the entities we want to extract, initializing with empty strings
    entities = {
        'Timestamp': current_time,
        'Companion': "",
        'Pain Relief': "",
        'Oral fluid': "",
        'Posture': "",
        'Baseline FHR': "",
        'FHR Decceleration': "",
        'Amniotic fluid': "",
        'Fetal position': "",
        'Caput': "",
        'Moulding': "",
        'Pulse': "",
        'Systolic BP': "",
        'Diastolic BP': "",
        'Temperature': "",
        'Urine': "",
        'Contractions per 10 min': "",
        'Duration of contractions': "",
        'Cervix': "",
        'Descent': "",
        'Oxytocin': "",
        'IV Fluid': "",
        'Medication': "",
        'Assessment': "",
        'Plan': ""
    }

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            entities['Companion'] = ent.text  # Extracting companion's name
        elif ent.label_ == "ORG" and "pain" in ent.text.lower():
            entities['Pain Relief'] = ent.text
        elif ent.label_ == "CARDINAL":
            if "contractions" in ent.sent.text.lower():
                entities['Contractions per 10 min'] = ent.text
            elif "seconds" in ent.sent.text.lower():
                entities['Duration of contractions'] = ent.text
            elif "fhr" in ent.sent.text.lower() or "heart rate" in ent.sent.text.lower():
                entities['Baseline FHR'] = ent.text
            elif "cm" in ent.sent.text.lower():
                entities['Cervix'] = ent.text

    # Extraction for specific phrases related to amniotic fluid, caput, moulding, fetal position, FHR deceleration, posture, companion status, pain relief, and oral fluid
    if "amniotic fluid" in doc.text.lower():
        if "intact" in doc.text.lower():
            entities['Amniotic fluid'] = "I"
        elif "clear" in doc.text.lower() and "ruptured" in doc.text.lower():
            entities['Amniotic fluid'] = "C"
        elif "meconium-stained" in doc.text.lower() and "ruptured" in doc.text.lower():
            # Check for thickness of meconium
            if "thick" in doc.text.lower():
                entities['Amniotic fluid'] = "M +++"
            elif "medium" in doc.text.lower():
                entities['Amniotic fluid'] = "M ++"
            else:
                entities['Amniotic fluid'] = "M +"  # Non-significant meconium
        elif "blood-stained" in doc.text.lower() and "ruptured" in doc.text.lower():
            entities['Amniotic fluid'] = "B"

    if "no caput" in doc.text.lower():
        entities['Caput'] = "0 (none)"
    elif "caput" in doc.text.lower():
        entities['Caput'] = "+ (present)"  # Assign + (present) when caput is mentioned without specifics

    if "no moulding" in doc.text.lower():
        entities['Moulding'] = "0 (none)"
    elif "moulding" in doc.text.lower():
        entities['Moulding'] = "+ (present)"  # Assign + (present) when moulding is mentioned without specifics

    # Fetal position extraction logic
    if "occiput anterior" in doc.text.lower():
        entities['Fetal position'] = "A"
    elif "occiput posterior" in doc.text.lower():
        entities['Fetal position'] = "P"
    elif "occiput transverse" in doc.text.lower():
        entities['Fetal position'] = "T"

    # FHR Deceleration extraction logic
    if "no deceleration" in doc.text.lower() or "without deceleration" in doc.text.lower():
        entities['FHR Decceleration'] = "N"
    elif "early deceleration" in doc.text.lower():
        entities['FHR Decceleration'] = "E"
    elif "late deceleration" in doc.text.lower():
        entities['FHR Decceleration'] = "L"
    elif "variable deceleration" in doc.text.lower():
        entities['FHR Decceleration'] = "V"

    # Posture extraction logic
    if "supine" in doc.text.lower():
        entities['Posture'] = "SP"
    elif any(term in doc.text.lower() for term in ["mobile", "walking", "swaying", "left lateral", "squatting", "kneeling", "standing"]):
        entities['Posture'] = "MO"

    # Companion status extraction logic
    if any(term in doc.text.lower() for term in ["companion present", "has a companion", "with companion"]):
        entities['Companion Status'] = "Y"
    elif any(term in doc.text.lower() for term in ["no companion", "without a companion"]):
        entities['Companion Status'] = "N"
    elif any(term in doc.text.lower() for term in ["declines companion", "woman declines"]):
        entities['Companion Status'] = "D"

    if 'alone' in text.lower():
        entities['Companion'] = 'N'
    elif 'with' in text.lower() and any(relative in text.lower() for relative in ['sister', 'mother', 'husband', 'partner']):
        entities['Companion'] = 'Y'

    # Pain relief status extraction logic
    if any(term in doc.text.lower() for term in ["receiving pain relief", "pain relief given", "pain medication"]):
        entities['Pain Relief'] = "Y"
    elif any(term in doc.text.lower() for term in ["not receiving pain relief", "no pain relief"]):
        entities['Pain Relief'] = "N"
    elif any(term in doc.text.lower() for term in ["declines pain relief", "woman declines pain relief"]):
        entities['Pain Relief'] = "D"

    # Oral fluid status extraction logic
    if any(term in doc.text.lower() for term in ["receiving oral fluids", "oral fluids given"]):
        entities['Oral fluid'] = "Y"
    elif any(term in doc.text.lower() for term in ["not receiving oral fluids", "no oral fluids"]):
        entities['Oral fluid'] = "N"
    elif any(term in doc.text.lower() for term in ["declines oral fluids", "woman declines oral fluids"]):
        entities['Oral fluid'] = "D"

    # Baseline FHR extraction logic: looking for a single number indicating FHR
    fhr_match = re.search(r'(\b\d{1,3}\b)(?=\s*beats? per minute)', text)
    if fhr_match:
        fhr_value = int(fhr_match.group(1))

        # Round to nearest 5 BPM increment
        rounded_fhr_value = round(fhr_value / 5) * 5

        # Ensure it falls within normal range (110-160 BPM)
        if rounded_fhr_value < 110 or rounded_fhr_value > 160:
            rounded_fhr_value = None

        entities['Baseline FHR'] = str(rounded_fhr_value) if rounded_fhr_value else None

    # Pulse extraction logic
    pulse_match = re.search(r'pulse rate is (\d+)', text.lower())
    if pulse_match:
        entities['Pulse'] = pulse_match.group(1)

    # Blood pressure extraction logic
    bp_match = re.search(r'blood pressure\s*(\d+)/(\d+)', text, re.IGNORECASE)
    if bp_match:
        entities['Systolic BP'] = bp_match.group(1)
        entities['Diastolic BP'] = bp_match.group(2)


    # Temperature processing
    temp_match = re.search(r'temperature is (\d+\.?\d*)', text.lower())
    if temp_match:
        entities['Temperature'] = temp_match.group(1)

    # Urine sample processing logic
    urine_results = []

    # Check for protein
    protein_match = re.search(r'protein\s*(?:[-:])?\s*(negative|trace|\+{1,4})', text.lower())
    if protein_match:
        urine_results.append(f"P: {protein_match.group(1).capitalize()}")
    elif "without proteinuria" in text.lower():
        urine_results.append("P: Negative")

    # Check for acetone
    acetone_match = re.search(r'acetone\s*(?:[-:])?\s*(negative|trace|\+{1,4})', text.lower())
    if acetone_match:
        urine_results.append(f"A: {acetone_match.group(1).capitalize()}")
    elif "without acetone" in text.lower():
        urine_results.append("A: Negative")

    # Combine results
    if urine_results:
        entities['Urine'] = ", ".join(urine_results)

    # Contractions per 10 min extraction logic
    contractions_match = re.search(r'(\w+|\d+)\s*(?:uterine\s*)?contractions?\s*(?:every|per)\s*10\s*minutes', text, re.IGNORECASE)
    if contractions_match:
        contractions = word_to_number(contractions_match.group(1))
        entities['Contractions per 10 min'] = int(contractions)

    # Duration of contractions extraction logic
    duration_match = re.search(r'lasting\s*(\d+)\s*seconds', text, re.IGNORECASE)
    if duration_match:
        entities['Duration of contractions'] = int(duration_match.group(1))

    # Cervix dilation extraction and interpretation
    cervix_match = re.search(r'(\d+)\s*cm\s*cervical\s*dilatation', text, re.IGNORECASE)
    if cervix_match:
        dilation = int(cervix_match.group(1))
        if dilation == 5:
            entities['Cervix'] = "≥6 h"
        elif dilation == 6:
            entities['Cervix'] = "≥5 h"
        elif dilation == 7:
            entities['Cervix'] = "≥3 h"
        elif dilation == 8:
            entities['Cervix'] = "≥2.5 h"
        elif dilation == 9:
            entities['Cervix'] = "≥2 h"
        else:
            entities['Cervix'] = f"{dilation} cm"

    # Descent extraction logic
    descent_match = re.search(r'Fetal descent is (\d)/5', text, re.IGNORECASE)
    if descent_match:
        entities['Descent'] = descent_match.group(1)

    # Oxytocin extraction logic
    oxytocin_match = re.search(r'oxytocin.*?(\d+)\s*U/L.*?(\d+)\s*drops/min', text, re.IGNORECASE | re.DOTALL)
    if oxytocin_match:
        entities['Oxytocin'] = f"{oxytocin_match.group(1)},{oxytocin_match.group(2)}"

    # IV Fluid extraction logic
    if 'iv fluid' in text.lower() or 'intravenous fluid' in text.lower():
        entities['IV Fluid'] = 'Y'

    # Medication extraction logic
    medication_match = re.search(r'(\w+)\s+(\d+\s*mg).*?(intramuscular|IM|intravenous|IV)', text, re.IGNORECASE)
    if medication_match:
        entities['Medication'] = f"{medication_match.group(1)} {medication_match.group(2)} {medication_match.group(3)}"

    return entities

# Test the function with an example text
text6 = ("Mrs. A, Para 0+0 presents in labor at 39 weeks , uneventful ANC, Contractions 1/10 each 20 seconds, Pulse 88, BP 130/80, FHR 140/min, "
"no decelerations, Head all up, Cervix mostly effaced, membranes intact, 4 cm dilatated. Assess the state of the patient.")
result6 = extract_relevant_info(text6)
print(result6)

{'Timestamp': '2024-12-07 04:00:01', 'Companion': 'A', 'Pain Relief': '', 'Oral fluid': '', 'Posture': '', 'Baseline FHR': '', 'FHR Decceleration': 'N', 'Amniotic fluid': '', 'Fetal position': '', 'Caput': '', 'Moulding': '', 'Pulse': '', 'Systolic BP': '', 'Diastolic BP': '', 'Temperature': '', 'Urine': '', 'Contractions per 10 min': '140', 'Duration of contractions': '', 'Cervix': '', 'Descent': '', 'Oxytocin': '', 'IV Fluid': '', 'Medication': '', 'Assessment': '', 'Plan': ''}


**Test Cases for NLP Processing based on WHO Labor Care Guide Manual**

In [ ]:
# Test Case 2
text2 = ("The baby moves during monitoring and shows a heart rate of 140 beats per minute (bpm). "
         "(Baseline FHR recorded). Vaginal examination shows 5 cm cervical dilatation, cephalic presentation. "
         "(Amniotic fluid: membranes ruptured, clear fluid.) There is no caput or moulding and the fetal position is occiput posterior. "
         "(The woman declines to receive pain relief and also declines oral fluids.)")
result2 = extract_relevant_info(text2)
print(result2)

# Test Case 3 (Pulse, SBP, DBP, Temperature, Urine)
text3 = ("Mary Jane's pulse rate is 88 bpm, with blood pressure of 120/80 mmHg. Her temperature is "
         "36.5°C. She passed urine at admission, without proteinuria or acetone. Given that all clinical "
         "woman parameters are normal, the midwife plans to reassess woman's observations in 4 hours "
         "unless otherwise indicated.")
result3 = extract_relevant_info(text3)
print(result3)

# Test Case 4 (Contractions per 10 min; Duration of Contractions; Cervix; Descent)
text4 = ("At the time of admission, Mary Jane presented with three uterine contractions every"
        "10 minutes, of moderate intensity, and lasting 40 seconds."
        "Vaginal examination shows 5 cm cervical dilatation, cephalic presentation. Fetal descent is 4/5.")
result4 = extract_relevant_info(text4)
print(result4)

# Test Case 5
text5 = ("Mary Jane complains of strong pains. Her sister left the labour ward and Mary Jane is"
"alone, lying in bed in a supine position. Her vitals are heart rate 96 bpm, blood pressure"
"128/84 mmHg, and FHR is 151 bpm with variable decelerations. Mary Jane has three strong uterine contractions in 10 minutes, lasting 50 seconds each. Fetal descent is 3/5. Cervical"
"dilatation is 8 cm and the fetal position is occiput transverse. Amniotic fluid shows meconium"
"1+/4.")
result5 = extract_relevant_info(text5)
print(result5)

{'Timestamp': '2024-12-07 04:00:13', 'Companion': '', 'Pain Relief': '', 'Oral fluid': 'D', 'Posture': '', 'Baseline FHR': '140', 'FHR Decceleration': '', 'Amniotic fluid': 'C', 'Fetal position': 'P', 'Caput': '0 (none)', 'Moulding': '+ (present)', 'Pulse': '', 'Systolic BP': '', 'Diastolic BP': '', 'Temperature': '', 'Urine': '', 'Contractions per 10 min': '', 'Duration of contractions': '', 'Cervix': '≥6 h', 'Descent': '', 'Oxytocin': '', 'IV Fluid': '', 'Medication': '', 'Assessment': '', 'Plan': '', 'Companion Status': 'D'}
{'Timestamp': '2024-12-07 04:00:13', 'Companion': "Mary Jane's", 'Pain Relief': '', 'Oral fluid': '', 'Posture': '', 'Baseline FHR': '', 'FHR Decceleration': '', 'Amniotic fluid': '', 'Fetal position': '', 'Caput': '', 'Moulding': '', 'Pulse': '88', 'Systolic BP': '', 'Diastolic BP': '', 'Temperature': '36.5', 'Urine': 'P: Negative', 'Contractions per 10 min': '', 'Duration of contractions': '', 'Cervix': '', 'Descent': '', 'Oxytocin': '', 'IV Fluid': '', 'Medic

**Alert System**

In [ ]:
def check_measurements_and_alert(data_entry):
    """
    Evaluate the parameters in a data entry and trigger alerts if thresholds are exceeded.

    :param data_entry: Dictionary containing the extracted measurement data.
    :return: List of alerts.
    """
    alerts = []

    # Thresholds and guidelines
    thresholds = {
        "Companion": {"N": "No companion available. Offer to find one for support."},
        "Pain Relief": {"N": "No pain relief administered. Offer pain relief based on preferences."},
        "Oral fluid": {"N": "Encourage the woman to drink fluids."},
        "Posture": {"SP": "Supine posture detected. Encourage mobility or non-supine positions."},
        "Baseline FHR": {"<110": "Slow FHR detected. Turn to left side and alert senior care.",
                         "≥160": "Rapid FHR detected. Turn to left side and alert senior care."},
        "FHR Decceleration": {"L": "Late deceleration detected. Perform prolonged auscultation and alert senior care."},
        "Amniotic fluid": {"M +++": "Thick meconium. Monitor closely and alert senior care.",
                           "B": "Blood-stained fluid. Monitor for complications and alert senior care."},
        "Fetal position": {"P": "Occiput posterior detected. Alert senior care.",
                           "T": "Occiput transverse detected. Alert senior care."},
        "Caput": {"+++": "Marked caput succedaneum. Alert senior care."},
        "Moulding": {"+++": "Severe moulding. Alert senior care."},
        "Pulse": {"<60": "Low pulse. Assess dehydration or complications.",
                  "≥120": "High pulse. Monitor for pain, fever, or shock."},
        "Systolic BP": {"<80": "Low systolic BP. Monitor for shock.",
                        "≥140": "High systolic BP. Monitor for hypertension."},
        "Diastolic BP": {"≥90": "High diastolic BP. Monitor for hypertension."},
        "Temperature": {"<35.0": "Low temperature. Monitor for hypothermia.",
                        "≥37.5": "High temperature. Monitor for fever or infection."},
        "Urine": {"P++": "Proteinuria detected. Monitor for pre-eclampsia.",
                  "A++": "Ketonuria detected. Monitor for dehydration or diabetes."},
        "Contractions per 10 min": {"≤2": "Inefficient contractions. Monitor closely.",
                                    ">5": "Excessive contractions. Alert senior care."},
        "Duration of contractions": {"<20": "Short contractions detected. Monitor closely.",
                                      ">60": "Prolonged contractions. Alert senior care."},
        "Cervix": {"5 cm ≥6 h": "No progress at 5 cm for 6 hours. Alert senior care.",
                   "6 cm ≥5 h": "No progress at 6 cm for 5 hours. Alert senior care.",
                   "7 cm ≥3 h": "No progress at 7 cm for 3 hours. Alert senior care.",
                   "8 cm ≥2.5 h": "No progress at 8 cm for 2.5 hours. Alert senior care.",
                   "9 cm ≥2 h": "No progress at 9 cm for 2 hours. Alert senior care."}
    }

    # Evaluate each parameter against thresholds
    for key, value in data_entry.items():
        if key in thresholds and value:
            for condition, alert_message in thresholds[key].items():
                # Numeric conditions
                if condition.startswith("<") or condition.startswith(">") or condition.startswith("≥") or condition.startswith("≤"):
                    operator = condition[:1]
                    threshold = float(condition[1:])
                    try:
                        if ((operator == "<" and float(value) < threshold) or
                            (operator == ">" and float(value) > threshold) or
                            (operator == "≥" and float(value) >= threshold) or
                            (operator == "≤" and float(value) <= threshold)):
                            alerts.append(f"{key}: {alert_message} (Value: {value})")
                    except ValueError:
                        pass  # Skip invalid comparisons

                # Categorical conditions
                elif value == condition:
                    alerts.append(f"{key}: {alert_message}")

    return alerts


In [ ]:
# Process text input to extract relevant information
data_entry = extract_relevant_info(text5)

# Check for alerts
alerts = check_measurements_and_alert(data_entry)

# Display alerts
if alerts:
    print("ALERTS:")
    for alert in alerts:
        print(alert)
else:
    print("No critical alerts detected.")


ALERTS:
Companion: No companion available. Offer to find one for support.
Posture: Supine posture detected. Encourage mobility or non-supine positions.
Fetal position: Occiput transverse detected. Alert senior care.
Duration of contractions: Short contractions detected. Monitor closely. (Value: 3)


In [ ]:
from datetime import datetime, timedelta

# Define monitoring frequencies for Stage 1 and Stage 2 (in minutes)
MONITORING_FREQUENCIES = {
    "Stage 1": {
        "Companion": 60,  # Hourly
        "Pain Relief": 60,
        "Oral Fluid": 60,
        "Posture": 60,
        "Baseline FHR": 30,
        "FHR Deceleration": 30,
        "Amniotic Fluid": 240,  # Every 4 hours
        "Fetal Position": 240,
        "Caput": 240,
        "Moulding": 240,
        "Pulse": 240,
        "Systolic BP": 240,
        "Diastolic BP": 240,
        "Temperature": 240,
        "Urine Protein": 240,
        "Urine Acetone": 240,
        "Contractions per 10 min": 30,
        "Duration of Contractions": 30,
        "Cervix Dilation": 240,
        "Descent": 240,
    },
    "Stage 2": {
        "Companion": 60,
        "Pain Relief": 60,
        "Oral Fluid": 60,
        "Posture": 60,
        "Baseline FHR": 15,  # Every 15 minutes
        "FHR Deceleration": 15,
        "Amniotic Fluid": 240,
        "Fetal Position": 240,
        "Caput": 240,
        "Moulding": 240,
        "Pulse": 240,
        "Systolic BP": 240,
        "Diastolic BP": 240,
        "Temperature": 240,
        "Urine Protein": 240,
        "Urine Acetone": 240,
        "Contractions per 10 min": 15,
        "Duration of Contractions": 15,
        "Descent": 15,
    }
}


In [ ]:
# Initialize a dictionary to track last recorded times for each parameter
last_recorded_times = {param: None for param in MONITORING_FREQUENCIES["Stage 1"]}

In [ ]:
def check_monitoring_frequencies(stage, current_time):
    """
    Check if parameters are being monitored at the required frequencies.

    :param stage: The labor stage ("Stage 1" or "Stage 2").
    :param current_time: The current datetime.
    :return: List of alerts for frequency violations.
    """
    alerts = []

    for parameter, frequency in MONITORING_FREQUENCIES[stage].items():
        last_time = last_recorded_times.get(parameter)

        # If this is the first recording, initialize the time
        if last_time is None:
            last_recorded_times[parameter] = current_time
            continue

        # Calculate the elapsed time
        elapsed_time = (current_time - last_time).total_seconds() / 60  # Convert to minutes

        # Check if the elapsed time exceeds the frequency
        if elapsed_time > frequency:
            alerts.append(f"Alert: {parameter} has not been monitored for {int(elapsed_time)} minutes. "
                          f"Expected every {frequency} minutes.")

    return alerts


In [ ]:
def update_last_recorded_times(parameter, current_time):
    """
    Update the last recorded time for a parameter.

    :param parameter: The parameter name.
    :param current_time: The current datetime.
    """
    last_recorded_times[parameter] = current_time


In [ ]:
def process_query(query, stage, history):
    current_time = datetime.now()

    # Extract information
    data_entry = extract_relevant_info(query)

    # Update last recorded times for all parameters in the query
    for parameter in data_entry.keys():
        if parameter in last_recorded_times:
            update_last_recorded_times(parameter, current_time)

    # Check for frequency violations
    frequency_alerts = check_monitoring_frequencies(stage, current_time)

    # Check parameter-specific alerts
    parameter_alerts = check_measurements_and_alert(data_entry)

    # Combine all alerts
    all_alerts = frequency_alerts + parameter_alerts

    # Log or display alerts
    if all_alerts:
        print("\n".join(all_alerts))
    else:
        print("No alerts.")


In [ ]:
query = "Baseline FHR 170, Pulse 130, Companion N"
stage = "Stage 1"  # or "Stage 2"
process_query(query, stage, history=[])


Companion: No companion available. Offer to find one for support.
